<a href="https://colab.research.google.com/github/rizkyjackbar/model_levelStres/blob/main/Fix_Model_Level_Stress_Prediction_With_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def load_dataset():
    df = pd.read_csv('/content/StressLevelDataset.csv')
    return df

# Fungsi untuk preprocessing data
def preprocess_data(df):

    # Memisahkan fitur dan target
    X = df.drop(columns=['stress_level'])  # Mengambil fitur tanpa kolom target
    y = df['stress_level']  # Target adalah kolom stress_level

    # Normalisasi fitur
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Membagi dataset menjadi training dan testing (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, scaler, X

# Fungsi untuk membuat dan melatih model neural network
def build_and_train_model(X_train, y_train):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Layer pertama
    model.add(Dense(32, activation='relu'))  # Layer kedua
    model.add(Dense(3, activation='softmax'))  # Layer output dengan 3 kelas (0, 1, 2)

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Melatih model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

    return model

# Fungsi untuk melakukan prediksi
def predict_stress(model, X_input):
    prediction = model.predict(X_input)
    predicted_class = np.argmax(prediction, axis=1)  # Mengambil kelas dengan probabilitas tertinggi
    return predicted_class[0]

# Fungsi untuk meminta input dari pengguna dengan validasi min-max
def get_user_input(df):
    user_input = []
    for col in df.columns:
        if col != 'stress_level':  # Menghindari input untuk kolom target
            min_value = df[col].min()
            max_value = df[col].max()

            while True:
                try:
                    value = float(input(f"Masukkan nilai untuk {col} (Rentang: {min_value} - {max_value}): "))
                    if min_value <= value <= max_value:
                        user_input.append(value)
                        break  # Keluar dari loop jika input valid
                    else:
                        print(f"Input tidak valid. Harap masukkan angka antara {min_value} dan {max_value}.")
                except ValueError:
                    print("Input tidak valid. Harap masukkan angka.")
    return user_input

# Fungsi utama untuk menjalankan program
def main():
    # Memuat dataset
    df = load_dataset()

    # Menampilkan kolom untuk debug
    print("Nama kolom dalam dataset:", df.columns)

    # Memastikan kolom 'stress_level' ada
    if 'stress_level' not in df.columns:
        print("Error: Kolom 'stress_level' tidak ditemukan.")
        return

    # Preprocessing data
    X_train, X_test, y_train, y_test, scaler, X = preprocess_data(df)

    # Membangun dan melatih model
    model = build_and_train_model(X_train, y_train)

    print("Model training selesai.")

    # Mengevaluasi model pada data test
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)  # Menentukan kelas yang diprediksi

    # Menampilkan akurasi
    accuracy = accuracy_score(y_test, y_pred_classes)
    print(f"Akurasi Model: {accuracy:.4f}")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_classes))

    # Input prediksi
    print("\nMasukkan data untuk prediksi tingkat stres:")
    user_input = get_user_input(df)

    # Normalisasi input user dengan scaler yang sudah digunakan untuk training
    user_input_scaled = scaler.transform([user_input])

    # Prediksi tingkat stres
    stress_level = predict_stress(model, user_input_scaled)

    # Menampilkan hasil prediksi
    if stress_level == 0:
        print("Tingkat Stres: Ringan")
    elif stress_level == 1:
        print("Tingkat Stres: Sedang")
    else:
        print("Tingkat Stres: Berat")

if __name__ == "__main__":
    main()


Nama kolom dalam dataset: Index(['anxiety_level', 'self_esteem', 'mental_health_history', 'depression',
       'headache', 'blood_pressure', 'sleep_quality', 'breathing_problem',
       'noise_level', 'living_conditions', 'safety', 'basic_needs',
       'academic_performance', 'study_load', 'teacher_student_relationship',
       'future_career_concerns', 'social_support', 'peer_pressure',
       'extracurricular_activities', 'bullying', 'stress_level'],
      dtype='object')
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5964 - loss: 0.9734
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8685 - loss: 0.5180 
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8810 - loss: 0.3348
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8766 - loss: 0.2836 
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8953 - loss: 0.2358
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9105 - loss: 0.2072
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9298 - loss: 0.1636
Epoch 8/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9150 - loss: 0.1952
Epoch 9/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9244 - loss: 0.1705
Epoch 10/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9332 - loss: 0.1595
Epoch 11/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9375 - loss: 0.1601
Epoch 12/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9529 - loss: 0.128

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Akurasi Model: 0.8500

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.86      0.84        76
           1       0.83      0.85      0.84        73
           2       0.90      0.85      0.87        71

    accuracy                           0.85       220
   macro avg       0.85      0.85      0.85       220
weighted avg       0.85      0.85      0.85       220


Masukkan data untuk prediksi tingkat stres:
Masukkan nilai untuk anxiety_level (Rentang: 0 - 21): 1
Masukkan nilai untuk self_esteem (Rentang: 0 - 30): 1
Masukkan nilai untuk mental_health_history (Rentang: 0 - 1): 1
Masukkan nilai untuk depression (Rentang: 0 - 27): 1
Masukkan nilai untuk headache (Rentang: 0 - 5): 
Input tidak valid. Harap masukkan angka.
Masukkan nilai untuk headache (Rentang: 0 - 5): 1
Masukkan nilai untuk blood_pressure (Rentang: 1 - 3): 1
Masukkan nilai untuk sleep_quality (Rentang: 0 - 5): 1
Masukkan nila

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
